# Hasil Eksperimen Sistem Pengenal Emosi Cakapan

## Library Preparation

In [2]:
!pip install arff

  Created wheel for arff: filename=arff-0.9-cp37-none-any.whl size=4970 sha256=73649febf2bd3e4d5fe184c0b4c987987db51887a993504477d77bf12a19ba3f
  Stored in directory: /root/.cache/pip/wheels/04/d0/70/2c73afedd3ac25c6085b528742c69b9587cbdfa67e5194583b
Successfully built arff


In [3]:
import arff
import joblib
from io import StringIO
from google.colab import drive
from google.colab import files

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from keras.models import Sequential
from keras import backend
from keras import layers
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [4]:
drive.mount('/content/gdrive')
drive_dir = "/content/gdrive/MyDrive/Teknik Informatika/Semester 7/TUGAS AKHIR/TA 2"

Mounted at /content/gdrive


## Load Data

### Audio

In [5]:
def read_arff_data(data):
  row = next(data)
  row = next(data)
  list_data = []
  i = 0
  while row:
    # if (i % 100 == 0):
    #   print('Reading...', i, 'data already done.')
    row_data = []
    for j in range(len(row)):
      if (j != 0 and row[j] != ''):
        row_data.append(row[j])
    list_data.append(row_data)
    try :
      row = next(data)
      i = i + 1
    except :
      break
  return np.array(list_data)

In [6]:
feature_akustik = read_arff_data(arff.load(drive_dir + '/Eksperimen/Data/eGeMAPS.arff'))
feature_akustik.shape

(10822, 88)

### Textual

In [7]:
data = joblib.load(drive_dir + '/Eksperimen/Data/stopword.pkl')
tfidf_vectorizer = TfidfVectorizer()
feature_tfidf = tfidf_vectorizer.fit_transform(data['Transkripsi'])
feature_leksikal = np.array(feature_tfidf.todense())
feature_leksikal.shape

(10822, 5017)

## Data Preparation

### Scaling Data Akustik

In [8]:
scaler = MinMaxScaler()
feature_akustik = scaler.fit_transform(feature_akustik)
feature_akustik.shape

(10822, 88)

### Construct Concatenate Features

In [9]:
def concat_features(feature_1, feature_2):
  features = []
  for i in range(len(feature_1)):
    temp = []
    temp.extend(feature_1[i])
    temp.extend(feature_2[i])
    features.append(temp)
  return np.array(features, dtype=object)

In [10]:
feature_concat = concat_features(feature_leksikal, feature_akustik)
feature_concat.shape

(10822, 5105)

### Construct Sequential Data 

In [11]:
def construct_sequential(id, data):
  sequential_datas = []
  new_data= []
  for i in range(len(id)):
    if id[i].endswith('001'):
      sequential_datas.append(new_data)
      new_data = []
    new_data.append(data[i])
  sequential_datas.append(new_data)

  return np.array(sequential_datas, dtype=object)[1:]

In [12]:
def construct_abstract(id, abstraksi):
  new_abstraksi = []
  for i in range(len(id)):
    if id[i].endswith('001'):
      new_abstraksi.append(abstraksi[i])

  return np.array(new_abstraksi, dtype=object)

In [13]:
id = data['ID Ucapan']
sequential_features_akustik, sequential_features_leksikal, sequential_features_concat = construct_sequential(id, feature_akustik), construct_sequential(id, feature_leksikal), construct_sequential(id, feature_concat)
print(sequential_features_akustik.shape, sequential_features_leksikal.shape, sequential_features_concat.shape)

(2003,) (2003,) (2003,)


In [14]:
abstraksi = construct_abstract(id, data['Abstraksi Emosi'])
print(abstraksi.shape)

(2003,)


### Label One Hot Encoding

In [15]:
encoder_ucapan = OneHotEncoder(sparse=False)
labels = np.array(data['Label Emosi']).reshape(-1, 1)
labels = encoder_ucapan.fit_transform(labels)
labels = construct_sequential(id, labels)
labels.shape

(2003,)

In [16]:
encoder_cakapan = OneHotEncoder(sparse=False)
abstraksi = abstraksi.reshape(-1, 1)
abstraksi = encoder_cakapan.fit_transform(abstraksi)
abstraksi.shape

(2003, 6)

### Split Train and Test Data

In [17]:
X_akustik = sequential_features_akustik
X_leksikal = sequential_features_leksikal
X_concat = sequential_features_concat
y_sequential = labels
y = abstraksi

In [18]:
X_akustik = tf.keras.preprocessing.sequence.pad_sequences(X_akustik, padding="post", dtype='float32')
X_leksikal = tf.keras.preprocessing.sequence.pad_sequences(X_leksikal, padding="post", dtype='float32')
X_concat = tf.keras.preprocessing.sequence.pad_sequences(X_concat, padding="post", dtype='float32')
y_sequential = tf.keras.preprocessing.sequence.pad_sequences(y_sequential, padding="post", dtype='float32')
y = tf.keras.preprocessing.sequence.pad_sequences(y, padding="post", dtype='float32')
print("Data", X_akustik.shape, X_leksikal.shape, X_concat.shape)
print("Label", y.shape, y_sequential.shape)

Data (2003, 15, 88) (2003, 15, 5017) (2003, 15, 5105)
Label (2003, 6) (2003, 15, 6)


In [19]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.15, random_state=42)
for train_index, test_index in sss.split(X_concat, y):
  train_index, test_index = train_index, test_index
print("Train data", train_index.shape)
print("Test data", test_index.shape)

Train data (1702,)
Test data (301,)


### Eliminate Minority Label

In [20]:
def get_label_idxs(list, label):
  idxs = []
  for i in range(len(list)):
    for l in label:
      if list[i] == l:
        idxs.append(i)
  return np.array(idxs)

In [21]:
def delete_elArr_by_idx(arr, idx):
  new_arr = []
  for i in range(len(arr)):
    if i not in idx:
      new_arr.append(arr[i])
  return np.array(new_arr)

In [22]:
dec_abstraksi = encoder_cakapan.inverse_transform(abstraksi)
del_idxs = get_label_idxs(dec_abstraksi, ["Jijik", "Takut"])

In [23]:
X_akustik_4 = np.delete(sequential_features_akustik, del_idxs)
X_leksikal_4 = np.delete(sequential_features_leksikal, del_idxs)
X_concat_4 = np.delete(sequential_features_concat, del_idxs)
label_4 = np.delete(dec_abstraksi, del_idxs)
print(X_akustik_4.shape, X_leksikal_4.shape, X_concat_4.shape, label_4.shape)

(1936,) (1936,) (1936,) (1936,)


In [24]:
encoder_cakapan_4 = OneHotEncoder(sparse=False)
label_4 = label_4.reshape(-1, 1)
label_4 = encoder_cakapan_4.fit_transform(label_4)
label_4.shape

(1936, 4)

In [25]:
X_akustik_4 = tf.keras.preprocessing.sequence.pad_sequences(X_akustik_4, padding="post", dtype='float32')
X_leksikal_4 = tf.keras.preprocessing.sequence.pad_sequences(X_leksikal_4, padding="post", dtype='float32')
X_concat_4 = tf.keras.preprocessing.sequence.pad_sequences(X_concat_4, padding="post", dtype='float32')
y_4 = tf.keras.preprocessing.sequence.pad_sequences(label_4, padding="post", dtype='float32')
print(X_akustik_4.shape, X_leksikal_4.shape, X_concat_4.shape, y_4.shape)

(1936, 15, 88) (1936, 15, 5017) (1936, 15, 5105) (1936, 4)


In [26]:
sss_4 = StratifiedShuffleSplit(n_splits=1, test_size=0.15, random_state=42)
for train_index_4, test_index_4 in sss_4.split(X_akustik_4, y_4):
  train_index_4, test_index_4 = train_index_4, test_index_4
print("Train data", train_index_4.shape)
print("Test data", test_index_4.shape)

Train data (1645,)
Test data (291,)


## 1. Model Langsung

### 1.1. Model Akustik

In [26]:
X_test, y_test = X_akustik[test_index], y[test_index]
model_akustik_langsung = keras.models.load_model(drive_dir + "/Eksperimen/Model/model_akustik_langsung_6.h5")
pred_akustik_langsung = model_akustik_langsung.predict(X_test)
y_true, y_pred_akustik_langsung = encoder_cakapan.inverse_transform(y_test), encoder_cakapan.inverse_transform(pred_akustik_langsung) 
print(classification_report(y_true, y_pred_akustik_langsung, digits=4))
print(confusion_matrix(y_true, y_pred_akustik_langsung))

              precision    recall  f1-score   support

       Jijik     0.0000    0.0000    0.0000         1
       Marah     0.3333    0.4286    0.3750        42
       Sedih     0.4176    0.4419    0.4294        86
      Senang     0.4286    0.5310    0.4743       113
       Takut     0.0000    0.0000    0.0000         9
    Terkejut     0.6875    0.2200    0.3333        50

    accuracy                         0.4219       301
   macro avg     0.3112    0.2702    0.2687       301
weighted avg     0.4409    0.4219    0.4084       301

[[ 0  0  0  1  0  0]
 [ 0 18  9 14  0  1]
 [ 0  9 38 39  0  0]
 [ 0 15 34 60  0  4]
 [ 0  1  3  5  0  0]
 [ 0 11  7 21  0 11]]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [27]:
X_test, y_test = X_akustik_4[test_index_4], y_4[test_index_4]
model_akustik_langsung_4 = keras.models.load_model(drive_dir + "/Eksperimen/Model/model_akustik_langsung_4.h5")
pred_akustik_langsung_4 = model_akustik_langsung_4.predict(X_test)
y_true_4, y_pred_akustik_langsung_4 = encoder_cakapan_4.inverse_transform(y_test), encoder_cakapan_4.inverse_transform(pred_akustik_langsung_4) 
print(classification_report(y_true_4, y_pred_akustik_langsung_4, digits=4))
print(confusion_matrix(y_true_4, y_pred_akustik_langsung_4))

              precision    recall  f1-score   support

       Marah     0.3462    0.4286    0.3830        42
       Sedih     0.3983    0.5465    0.4608        86
      Senang     0.4494    0.3540    0.3960       113
    Terkejut     0.5312    0.3400    0.4146        50

    accuracy                         0.4192       291
   macro avg     0.4313    0.4173    0.4136       291
weighted avg     0.4335    0.4192    0.4165       291

[[18 12 10  2]
 [ 8 47 29  2]
 [15 47 40 11]
 [11 12 10 17]]


### 1.2. Model Leksikal

In [28]:
X_test, y_test = X_leksikal[test_index], y[test_index]
model_leksikal_langsung = keras.models.load_model(drive_dir + "/Eksperimen/Model/model_leksikal_langsung_6.h5")
pred_leksikal_langsung = model_leksikal_langsung.predict(X_test)
y_true, y_pred_leksikal_langsung = encoder_cakapan.inverse_transform(y_test), encoder_cakapan.inverse_transform(pred_leksikal_langsung) 
print(classification_report(y_true, y_pred_leksikal_langsung, digits=4))
print(confusion_matrix(y_true, y_pred_leksikal_langsung))

              precision    recall  f1-score   support

       Jijik     0.0000    0.0000    0.0000         1
       Marah     0.7083    0.4048    0.5152        42
       Sedih     0.4227    0.4767    0.4481        86
      Senang     0.5796    0.8053    0.6741       113
       Takut     0.0000    0.0000    0.0000         9
    Terkejut     0.6087    0.2800    0.3836        50

    accuracy                         0.5415       301
   macro avg     0.3866    0.3278    0.3368       301
weighted avg     0.5383    0.5415    0.5167       301

[[ 0  0  1  0  0  0]
 [ 0 17 14 10  0  1]
 [ 0  2 41 39  0  4]
 [ 0  0 19 91  0  3]
 [ 0  1  6  1  0  1]
 [ 0  4 16 16  0 14]]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [29]:
X_test, y_test = X_leksikal_4[test_index_4], y_4[test_index_4]
model_leksikal_langsung_4 = keras.models.load_model(drive_dir + "/Eksperimen/Model/model_leksikal_langsung_4.h5")
pred_leksikal_langsung_4 = model_leksikal_langsung_4.predict(X_test)
y_true_4, y_pred_leksikal_langsung_4 = encoder_cakapan_4.inverse_transform(y_test), encoder_cakapan_4.inverse_transform(pred_leksikal_langsung_4) 
print(classification_report(y_true_4, y_pred_leksikal_langsung_4, digits=4))
print(confusion_matrix(y_true_4, y_pred_leksikal_langsung_4))

              precision    recall  f1-score   support

       Marah     0.5526    0.5000    0.5250        42
       Sedih     0.4375    0.4070    0.4217        86
      Senang     0.6202    0.7080    0.6612       113
    Terkejut     0.5455    0.4800    0.5106        50

    accuracy                         0.5498       291
   macro avg     0.5389    0.5237    0.5296       291
weighted avg     0.5436    0.5498    0.5449       291

[[21 13  5  3]
 [ 8 35 34  9]
 [ 3 22 80  8]
 [ 6 10 10 24]]


### 1.3. Model Gabungan Level Keputusan

In [30]:
def get_prediction_gabungan(pred_akustik, pred_leksikal):
  pred_gabungan = []
  for i in range(len(pred_akustik)):
    if (pred_akustik[i].max() > pred_leksikal[i].max()):
      pred_gabungan.append(pred_akustik[i])
    else :
      pred_gabungan.append(pred_leksikal[i])
  pred_gabungan = np.array(pred_gabungan)
  return pred_gabungan

In [31]:
y_test = y[test_index]
pred_gabungan_langsung_lv_keputusan = get_prediction_gabungan(pred_akustik_langsung, pred_leksikal_langsung)
y_true,y_pred_gabungan_langsung_lv_keputusan = encoder_cakapan.inverse_transform(y_test), encoder_cakapan.inverse_transform(pred_gabungan_langsung_lv_keputusan) 
print(classification_report(y_true, y_pred_gabungan_langsung_lv_keputusan, digits=4))
print(confusion_matrix(y_true, y_pred_gabungan_langsung_lv_keputusan))

              precision    recall  f1-score   support

       Jijik     0.0000    0.0000    0.0000         1
       Marah     0.4242    0.3333    0.3733        42
       Sedih     0.4286    0.4186    0.4235        86
      Senang     0.5116    0.7788    0.6175       113
       Takut     0.0000    0.0000    0.0000         9
    Terkejut     0.7500    0.1800    0.2903        50

    accuracy                         0.4884       301
   macro avg     0.3524    0.2851    0.2841       301
weighted avg     0.4983    0.4884    0.4532       301

[[ 0  0  0  1  0  0]
 [ 0 14 14 13  0  1]
 [ 0  5 36 45  0  0]
 [ 0  4 19 88  0  2]
 [ 0  1  4  4  0  0]
 [ 0  9 11 21  0  9]]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [32]:
y_test = y_4[test_index_4]
pred_gabungan_langsung_lv_keputusan_4 = get_prediction_gabungan(pred_akustik_langsung_4, pred_leksikal_langsung_4)
y_true_4, y_pred_gabungan_keputusan_lv_fitur_4 = encoder_cakapan_4.inverse_transform(y_test), encoder_cakapan_4.inverse_transform(pred_gabungan_langsung_lv_keputusan_4) 
print(classification_report(y_true_4, y_pred_gabungan_keputusan_lv_fitur_4, digits=4))
print(confusion_matrix(y_true_4, y_pred_gabungan_keputusan_lv_fitur_4))

              precision    recall  f1-score   support

       Marah     0.5500    0.5238    0.5366        42
       Sedih     0.4304    0.3953    0.4121        86
      Senang     0.6107    0.7080    0.6557       113
    Terkejut     0.6341    0.5200    0.5714        50

    accuracy                         0.5567       291
   macro avg     0.5563    0.5368    0.5440       291
weighted avg     0.5527    0.5567    0.5521       291

[[22 12  5  3]
 [ 9 34 37  6]
 [ 4 23 80  6]
 [ 5 10  9 26]]


### 1.4. Model Gabungan Level Fitur

In [33]:
X_test, y_test = X_concat[test_index], y[test_index]
model_gabungan_langsung = keras.models.load_model(drive_dir + "/Eksperimen/Model/model_gabungan_langsung_6.h5")
pred_gabungan_langsung_lv_fitur = model_gabungan_langsung.predict(X_test)
y_true, y_pred_gabungan_langsung_lv_fitur = encoder_cakapan.inverse_transform(y_test), encoder_cakapan.inverse_transform(pred_gabungan_langsung_lv_fitur) 
print(classification_report(y_true, y_pred_gabungan_langsung_lv_fitur, digits=4))
print(confusion_matrix(y_true, y_pred_gabungan_langsung_lv_fitur))

              precision    recall  f1-score   support

       Jijik     0.0000    0.0000    0.0000         1
       Marah     0.6129    0.4524    0.5205        42
       Sedih     0.4795    0.4070    0.4403        86
      Senang     0.5745    0.7168    0.6378       113
       Takut     0.0000    0.0000    0.0000         9
    Terkejut     0.5179    0.5800    0.5472        50

    accuracy                         0.5449       301
   macro avg     0.3641    0.3594    0.3576       301
weighted avg     0.5242    0.5449    0.5288       301

[[ 0  0  0  0  0  1]
 [ 0 19  9  8  0  6]
 [ 0  8 35 36  0  7]
 [ 0  1 20 81  0 11]
 [ 0  1  4  2  0  2]
 [ 0  2  5 14  0 29]]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [34]:
X_test, y_test = X_concat_4[test_index_4], y_4[test_index_4]
model_gabungan_langsung_4 = keras.models.load_model(drive_dir + "/Eksperimen/Model/model_gabungan_langsung_4.h5")
pred_gabungan_langsung_lv_fitur_4 = model_gabungan_langsung_4.predict(X_test)
y_true_4, y_pred_gabungan_langsung_lv_fitur_4 = encoder_cakapan_4.inverse_transform(y_test), encoder_cakapan_4.inverse_transform(pred_gabungan_langsung_lv_fitur_4) 
print(classification_report(y_true_4, y_pred_gabungan_langsung_lv_fitur_4, digits=4))
print(confusion_matrix(y_true_4, y_pred_gabungan_langsung_lv_fitur_4))

              precision    recall  f1-score   support

       Marah     0.6061    0.4762    0.5333        42
       Sedih     0.5062    0.4767    0.4910        86
      Senang     0.6325    0.6549    0.6435       113
    Terkejut     0.5333    0.6400    0.5818        50

    accuracy                         0.5739       291
   macro avg     0.5695    0.5620    0.5624       291
weighted avg     0.5743    0.5739    0.5719       291

[[20 12  4  6]
 [ 5 41 29 11]
 [ 5 23 74 11]
 [ 3  5 10 32]]


### Hasil Pengenalan Emosi Cakapan

In [35]:
pred_akustik_proba = pred_akustik_langsung.max(axis=1)
pred_leksikal_proba = pred_leksikal_langsung.max(axis=1)
pred_gabungan_fitur_proba = pred_gabungan_langsung_lv_fitur.max(axis=1)
pred_gabungan_keputusan_proba = pred_gabungan_langsung_lv_keputusan.max(axis=1)

pred_akustik = encoder_cakapan.inverse_transform(pred_akustik_langsung).reshape(301,)
pred_leksikal = encoder_cakapan.inverse_transform(pred_leksikal_langsung).reshape(301,)
pred_gabungan_fitur = encoder_cakapan.inverse_transform(pred_gabungan_langsung_lv_fitur).reshape(301,)
pred_gabungan_keputusan = encoder_cakapan.inverse_transform(pred_gabungan_langsung_lv_keputusan).reshape(301,)
true = encoder_cakapan.inverse_transform(y[test_index]).reshape(301,)

result = {
          'Abstraksi Emosi' : true, 
          'Prediksi Model Akustik' : pred_akustik,
          'Confidence Level Model Akustik' : pred_akustik_proba,
          'Prediksi Model Leksikal' : pred_leksikal,
          'Confidence Level Model Leksikal' : pred_leksikal_proba,
          'Prediksi Model Gabungan Level Fitur' : pred_gabungan_fitur,
          'Confidence Level Model Gabungan Level Fitur' : pred_gabungan_fitur_proba,
          'Prediksi Model Gabungan Level Keputusan' : pred_gabungan_keputusan,
          'Confidence Level Model Gabungan Level Keputusan' : pred_gabungan_keputusan_proba,
          }
df = pd.DataFrame(result)
df.to_excel("result_langsung_6.xlsx") 
files.download("result_langsung_6.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [36]:
pred_akustik_proba = pred_akustik_langsung_4.max(axis=1)
pred_leksikal_proba = pred_leksikal_langsung_4.max(axis=1)
pred_gabungan_fitur_proba = pred_gabungan_langsung_lv_fitur_4.max(axis=1)
pred_gabungan_keputusan_proba = pred_gabungan_langsung_lv_keputusan_4.max(axis=1)

pred_akustik = encoder_cakapan_4.inverse_transform(pred_akustik_langsung_4).reshape(291,)
pred_leksikal = encoder_cakapan_4.inverse_transform(pred_leksikal_langsung_4).reshape(291,)
pred_gabungan_fitur = encoder_cakapan_4.inverse_transform(pred_gabungan_langsung_lv_fitur_4).reshape(291,)
pred_gabungan_keputusan = encoder_cakapan_4.inverse_transform(pred_gabungan_langsung_lv_keputusan_4).reshape(291,)
true = encoder_cakapan_4.inverse_transform(y_4[test_index_4]).reshape(291,)

result = {
          'Abstraksi Emosi' : true, 
          'Prediksi Model Akustik' : pred_akustik,
          'Confidence Level Model Akustik' : pred_akustik_proba,
          'Prediksi Model Leksikal' : pred_leksikal,
          'Confidence Level Model Leksikal' : pred_leksikal_proba,
          'Prediksi Model Gabungan Level Fitur' : pred_gabungan_fitur,
          'Confidence Level Model Gabungan Level Fitur' : pred_gabungan_fitur_proba,
          'Prediksi Model Gabungan Level Keputusan' : pred_gabungan_keputusan,
          'Confidence Level Model Gabungan Level Keputusan' : pred_gabungan_keputusan_proba,
          }
df = pd.DataFrame(result)
df.to_excel("result_langsung_4.xlsx") 
files.download("result_langsung_4.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 2. Model Bertahap

In [27]:
def extract_contextual_features(model, idx_output, data):
  get_relu_output = backend.function([model.layers[0].input], [model.layers[idx_output].output])
  relu_output = get_relu_output([data])
  return np.array(relu_output[0])

### 2.1. Model Akustik

In [30]:
model_akustik_contextual = keras.models.load_model(drive_dir + "/Eksperimen/Model/model_akustik_bertahap_contextual.h5")
model_akustik_contextual.evaluate(X_akustik[test_index], y_sequential[test_index])

10/10 [==============================] - 1s 9ms/step - loss: 0.4750 - accuracy: 0.3997


[0.4750272035598755, 0.39974457025527954]

In [31]:
contextual_feature_akustik = joblib.load(drive_dir + "/Eksperimen/Data/contextual_feature_akustik.pkl")
X_test, y_test = contextual_feature_akustik[test_index], y[test_index]
model_akustik_bertahap = keras.models.load_model(drive_dir + "/Eksperimen/Model/model_akustik_bertahap_classification_6.h5")
pred_akustik_bertahap = model_akustik_bertahap.predict(X_test)
y_true, y_pred_akustik_bertahap = encoder_cakapan.inverse_transform(y_test), encoder_cakapan.inverse_transform(pred_akustik_bertahap) 
print(classification_report(y_true, y_pred_akustik_bertahap, digits=4))
print(confusion_matrix(y_true, y_pred_akustik_bertahap))

              precision    recall  f1-score   support

       Jijik     0.0000    0.0000    0.0000         1
       Marah     0.4000    0.4286    0.4138        42
       Sedih     0.3974    0.3605    0.3780        86
      Senang     0.4476    0.5664    0.5000       113
       Takut     0.0000    0.0000    0.0000         9
    Terkejut     0.4571    0.3200    0.3765        50

    accuracy                         0.4286       301
   macro avg     0.2837    0.2792    0.2781       301
weighted avg     0.4133    0.4286    0.4160       301

[[ 0  0  0  0  0  1]
 [ 0 18  9 11  0  4]
 [ 0  9 31 43  0  3]
 [ 0 11 27 64  0 11]
 [ 0  1  2  6  0  0]
 [ 0  6  9 19  0 16]]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [32]:
contextual_feature_akustik_4 = joblib.load(drive_dir + "/Eksperimen/Data/contextual_feature_akustik_4.pkl")
X_test, y_test = contextual_feature_akustik_4[test_index_4], y_4[test_index_4]
model_akustik_bertahap_4 = keras.models.load_model(drive_dir + "/Eksperimen/Model/model_akustik_bertahap_classification_4.h5")
pred_akustik_bertahap_4 = model_akustik_bertahap_4.predict(X_test)
y_true_4, y_pred_akustik_bertahap_4 = encoder_cakapan_4.inverse_transform(y_test), encoder_cakapan_4.inverse_transform(pred_akustik_bertahap_4) 
print(classification_report(y_true_4, y_pred_akustik_bertahap_4, digits=4))
print(confusion_matrix(y_true_4, y_pred_akustik_bertahap_4))

              precision    recall  f1-score   support

       Marah     0.4667    0.3333    0.3889        42
       Sedih     0.4409    0.4767    0.4581        86
      Senang     0.4552    0.5841    0.5116       113
    Terkejut     0.4348    0.2000    0.2740        50

    accuracy                         0.4502       291
   macro avg     0.4494    0.3985    0.4081       291
weighted avg     0.4491    0.4502    0.4373       291

[[14 11 15  2]
 [ 6 41 37  2]
 [ 7 31 66  9]
 [ 3 10 27 10]]


### 2.2. Model Leksikal

In [33]:
model_leksikal_contextual = keras.models.load_model(drive_dir + "/Eksperimen/Model/model_leksikal_bertahap_contextual.h5")
model_leksikal_contextual.evaluate(X_leksikal[test_index], y_sequential[test_index])

10/10 [==============================] - 1s 22ms/step - loss: 0.4384 - accuracy: 0.5051


[0.43836164474487305, 0.5051085352897644]

In [34]:
X_test, y_test = extract_contextual_features(model_leksikal_contextual, 3, X_leksikal)[test_index], y[test_index]
model_leksikal_bertahap = keras.models.load_model(drive_dir + "/Eksperimen/Model/model_leksikal_bertahap_classification_6.h5")
pred_leksikal_bertahap = model_leksikal_bertahap.predict(X_test)
y_true, y_pred_leksikal_bertahap = encoder_cakapan.inverse_transform(y_test), encoder_cakapan.inverse_transform(pred_leksikal_bertahap) 
print(classification_report(y_true, y_pred_leksikal_bertahap, digits=4))
print(confusion_matrix(y_true, y_pred_leksikal_bertahap))

              precision    recall  f1-score   support

       Jijik     0.0000    0.0000    0.0000         1
       Marah     0.5667    0.4048    0.4722        42
       Sedih     0.4907    0.6163    0.5464        86
      Senang     0.6525    0.6814    0.6667       113
       Takut     0.0000    0.0000    0.0000         9
    Terkejut     0.5556    0.5000    0.5263        50

    accuracy                         0.5714       301
   macro avg     0.3776    0.3671    0.3686       301
weighted avg     0.5565    0.5714    0.5597       301

[[ 0  0  1  0  0  0]
 [ 0 17 18  3  0  4]
 [ 0  6 53 21  0  6]
 [ 0  2 25 77  0  9]
 [ 0  1  4  3  0  1]
 [ 0  4  7 14  0 25]]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [35]:
contextual_feature_leksikal_4 = delete_elArr_by_idx(extract_contextual_features(model_leksikal_contextual, 3, X_leksikal), del_idxs)
X_test, y_test = contextual_feature_leksikal_4[test_index_4], y_4[test_index_4]
model_leksikal_bertahap_4 = keras.models.load_model(drive_dir + "/Eksperimen/Model/model_leksikal_bertahap_classification_4.h5")
pred_leksikal_bertahap_4 = model_leksikal_bertahap_4.predict(X_test)
y_true_4, y_pred_leksikal_bertahap_4 = encoder_cakapan_4.inverse_transform(y_test), encoder_cakapan_4.inverse_transform(pred_leksikal_bertahap_4) 
print(classification_report(y_true_4, y_pred_leksikal_bertahap_4, digits=4))
print(confusion_matrix(y_true_4, y_pred_leksikal_bertahap_4))

              precision    recall  f1-score   support

       Marah     0.6735    0.7857    0.7253        42
       Sedih     0.6914    0.6512    0.6707        86
      Senang     0.7120    0.7876    0.7479       113
    Terkejut     0.6667    0.4800    0.5581        50

    accuracy                         0.6942       291
   macro avg     0.6859    0.6761    0.6755       291
weighted avg     0.6925    0.6942    0.6892       291

[[33  5  2  2]
 [ 7 56 20  3]
 [ 5 12 89  7]
 [ 4  8 14 24]]


### 2.3. Model Gabungan Level Keputusan

In [36]:
def get_prediction_gabungan(pred_akustik, pred_leksikal):
  pred_gabungan = []
  for i in range(len(pred_akustik)):
    if (pred_akustik[i].max() > pred_leksikal[i].max()):
      pred_gabungan.append(pred_akustik[i])
    else :
      pred_gabungan.append(pred_leksikal[i])
  pred_gabungan = np.array(pred_gabungan)
  return pred_gabungan

In [37]:
y_test = y[test_index]
pred_gabungan_bertahap_lv_keputusan = get_prediction_gabungan(pred_akustik_bertahap, pred_leksikal_bertahap)
y_true, y_pred_gabungan_bertahap_lv_keputusan = encoder_cakapan.inverse_transform(y_test), encoder_cakapan.inverse_transform(pred_gabungan_bertahap_lv_keputusan) 
print(classification_report(y_true, y_pred_gabungan_bertahap_lv_keputusan, digits=4))
print(confusion_matrix(y_true, y_pred_gabungan_bertahap_lv_keputusan))

              precision    recall  f1-score   support

       Jijik     0.0000    0.0000    0.0000         1
       Marah     0.5758    0.4524    0.5067        42
       Sedih     0.4811    0.5930    0.5313        86
      Senang     0.6387    0.6726    0.6552       113
       Takut     0.0000    0.0000    0.0000         9
    Terkejut     0.5814    0.5000    0.5376        50

    accuracy                         0.5681       301
   macro avg     0.3795    0.3697    0.3718       301
weighted avg     0.5541    0.5681    0.5578       301

[[ 0  0  1  0  0  0]
 [ 0 19 18  3  0  2]
 [ 0  6 51 23  0  6]
 [ 0  2 26 76  0  9]
 [ 0  1  4  3  0  1]
 [ 0  5  6 14  0 25]]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [38]:
y_test = y_4[test_index_4]
pred_gabungan_bertahap_lv_keputusan_4 = get_prediction_gabungan(pred_akustik_bertahap_4, pred_leksikal_bertahap_4)
y_true_4, y_pred_gabungan_bertahap_lv_keputusan_4 = encoder_cakapan_4.inverse_transform(y_test), encoder_cakapan_4.inverse_transform(pred_gabungan_bertahap_lv_keputusan_4) 
print(classification_report(y_true_4, y_pred_gabungan_bertahap_lv_keputusan_4, digits=4))
print(confusion_matrix(y_true_4, y_pred_gabungan_bertahap_lv_keputusan_4))

              precision    recall  f1-score   support

       Marah     0.6800    0.8095    0.7391        42
       Sedih     0.6914    0.6512    0.6707        86
      Senang     0.7177    0.7876    0.7511       113
    Terkejut     0.6667    0.4800    0.5581        50

    accuracy                         0.6976       291
   macro avg     0.6889    0.6821    0.6797       291
weighted avg     0.6957    0.6976    0.6924       291

[[34  5  1  2]
 [ 7 56 20  3]
 [ 5 12 89  7]
 [ 4  8 14 24]]


### 2.4. Model Gabungan Level Fitur

In [39]:
model_gabungan_contextual = keras.models.load_model(drive_dir + "/Eksperimen/Model/model_gabungan_bertahap_contextual.h5")
model_gabungan_contextual.evaluate(X_concat[test_index], y_sequential[test_index])

10/10 [==============================] - 2s 76ms/step - loss: 0.5124 - accuracy: 0.5192


[0.5123841166496277, 0.5191571116447449]

In [40]:
contextual_features_concat = joblib.load(drive_dir + "/Eksperimen/Data/contextual_features_concat.pkl")
X_test, y_test = contextual_features_concat[test_index], y[test_index]
model_gabungan_bertahap = keras.models.load_model(drive_dir + "/Eksperimen/Model/model_gabungan_bertahap_classification_6.h5")
pred_gabungan_bertahap_lv_fitur = model_gabungan_bertahap.predict(X_test)
y_true, y_pred_gabungan_bertahap_lv_fitur = encoder_cakapan.inverse_transform(y_test), encoder_cakapan.inverse_transform(pred_gabungan_bertahap_lv_fitur) 
print(classification_report(y_true, y_pred_gabungan_bertahap_lv_fitur, digits=4))
print(confusion_matrix(y_true, y_pred_gabungan_bertahap_lv_fitur))

              precision    recall  f1-score   support

       Jijik     0.0000    0.0000    0.0000         1
       Marah     0.5938    0.4524    0.5135        42
       Sedih     0.5192    0.6279    0.5684        86
      Senang     0.6726    0.6726    0.6726       113
       Takut     0.5000    0.1111    0.1818         9
    Terkejut     0.5400    0.5400    0.5400        50

    accuracy                         0.5880       301
   macro avg     0.4709    0.4007    0.4127       301
weighted avg     0.5883    0.5880    0.5817       301

[[ 0  0  1  0  0  0]
 [ 0 19 16  3  0  4]
 [ 0  3 54 22  0  7]
 [ 0  3 22 76  0 12]
 [ 0  1  4  3  1  0]
 [ 0  6  7  9  1 27]]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [41]:
contextual_features_concat_4 = joblib.load(drive_dir + "/Eksperimen/Data/contextual_features_concat_4.pkl")
X_test, y_test = contextual_features_concat_4[test_index_4], y_4[test_index_4]
model_gabungan_bertahap_4 = keras.models.load_model(drive_dir + "/Eksperimen/Model/model_gabungan_bertahap_classification_4.h5")
pred_gabungan_bertahap_lv_fitur_4 = model_gabungan_bertahap_4.predict(X_test)
y_true_4, y_pred_gabungan_bertahap_lv_fitur_4 = encoder_cakapan_4.inverse_transform(y_test), encoder_cakapan_4.inverse_transform(pred_gabungan_bertahap_lv_fitur_4) 
print(classification_report(y_true_4, y_pred_gabungan_bertahap_lv_fitur_4, digits=4))
print(confusion_matrix(y_true_4, y_pred_gabungan_bertahap_lv_fitur_4))

              precision    recall  f1-score   support

       Marah     0.9143    0.7619    0.8312        42
       Sedih     0.7126    0.7209    0.7168        86
      Senang     0.7593    0.7257    0.7421       113
    Terkejut     0.5574    0.6800    0.6126        50

    accuracy                         0.7216       291
   macro avg     0.7359    0.7221    0.7257       291
weighted avg     0.7332    0.7216    0.7252       291

[[32  4  2  4]
 [ 1 62 17  6]
 [ 1 13 82 17]
 [ 1  8  7 34]]


### Hasil Pengenalan Emosi Cakapan

In [50]:
pred_akustik_proba = pred_akustik_bertahap.max(axis=1)
pred_leksikal_proba = pred_leksikal_bertahap.max(axis=1)
pred_gabungan_fitur_proba = pred_gabungan_bertahap_lv_fitur.max(axis=1)
pred_gabungan_keputusan_proba = pred_gabungan_bertahap_lv_keputusan.max(axis=1)

pred_akustik = encoder_cakapan.inverse_transform(pred_akustik_bertahap).reshape(301,)
pred_leksikal = encoder_cakapan.inverse_transform(pred_leksikal_bertahap).reshape(301,)
pred_gabungan_fitur = encoder_cakapan.inverse_transform(pred_gabungan_bertahap_lv_fitur).reshape(301,)
pred_gabungan_keputusan = encoder_cakapan.inverse_transform(pred_gabungan_bertahap_lv_keputusan).reshape(301,)
true = encoder_cakapan.inverse_transform(y[test_index]).reshape(301,)

result = {
          'Abstraksi Emosi' : true, 
          'Prediksi Model Akustik' : pred_akustik,
          'Confidence Level Model Akustik' : pred_akustik_proba,
          'Prediksi Model Leksikal' : pred_leksikal,
          'Confidence Level Model Leksikal' : pred_leksikal_proba,
          'Prediksi Model Gabungan Level Fitur' : pred_gabungan_fitur,
          'Confidence Level Model Gabungan Level Fitur' : pred_gabungan_fitur_proba,
          'Prediksi Model Gabungan Level Keputusan' : pred_gabungan_keputusan,
          'Confidence Level Model Gabungan Level Keputusan' : pred_gabungan_keputusan_proba,
          }
df = pd.DataFrame(result)
df.to_excel("result_bertahap_6.xlsx") 
files.download("result_bertahap_6.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [51]:
pred_akustik_proba = pred_akustik_bertahap_4.max(axis=1)
pred_leksikal_proba = pred_leksikal_bertahap_4.max(axis=1)
pred_gabungan_fitur_proba = pred_gabungan_bertahap_lv_fitur_4.max(axis=1)
pred_gabungan_keputusan_proba = pred_gabungan_bertahap_lv_keputusan_4.max(axis=1)

pred_akustik = encoder_cakapan_4.inverse_transform(pred_akustik_bertahap_4).reshape(291,)
pred_leksikal = encoder_cakapan_4.inverse_transform(pred_leksikal_bertahap_4).reshape(291,)
pred_gabungan_fitur = encoder_cakapan_4.inverse_transform(pred_gabungan_bertahap_lv_fitur_4).reshape(291,)
pred_gabungan_keputusan = encoder_cakapan_4.inverse_transform(pred_gabungan_bertahap_lv_keputusan_4).reshape(291,)
true = encoder_cakapan_4.inverse_transform(y_4[test_index_4]).reshape(291,)

result = {
          'Abstraksi Emosi' : true, 
          'Prediksi Model Akustik' : pred_akustik,
          'Confidence Level Model Akustik' : pred_akustik_proba,
          'Prediksi Model Leksikal' : pred_leksikal,
          'Confidence Level Model Leksikal' : pred_leksikal_proba,
          'Prediksi Model Gabungan Level Fitur' : pred_gabungan_fitur,
          'Confidence Level Model Gabungan Level Fitur' : pred_gabungan_fitur_proba,
          'Prediksi Model Gabungan Level Keputusan' : pred_gabungan_keputusan,
          'Confidence Level Model Gabungan Level Keputusan' : pred_gabungan_keputusan_proba,
          }
df = pd.DataFrame(result)
df.to_excel("result_bertahap_4.xlsx") 
files.download("result_bertahap_4.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>